In [1]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 5.5MB 20.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=08f1b1a86d9abe3854a6b270b4ec502a5ac4eb58caadf8533ebf87f6cafd3c04
  Stored in directory: /tmp/pip-ephem-wheel-cache-urwt_sv2/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import yfinance as yf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
import keras

In [3]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
Tickers = df['Symbol'].to_numpy().tolist()
i1=Tickers.index('BF.B')
i2=Tickers.index('BRK.B')
Tickers[i1]='BF-B'
Tickers[i2]='BRK-B'

In [4]:
df =  yf.download(tickers=Tickers, start="2019-12-31", end="2020-12-31")

[*********************100%***********************]  505 of 505 completed


In [5]:
# Take the Adjusted Returns from the data for the companies that are in the list for at least 10 years
dff=df['Adj Close']
df1 = dff[1:]
df0 = dff[:-1]
pt1 = df1.reset_index(drop=True)
pt0 = df0.reset_index(drop=True)
ret = (pt1-pt0)/pt0
ret.index = df1.index
ret.drop(ret.index[0],inplace=True)
lst=[]
for col in ret.columns:
    if ret[col].isna().sum() > 0:
        lst.append(col)
ret.drop(columns=lst,inplace=True)

In [6]:
ret.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.007502,0.014296,-0.004683,0.022816,0.011407,0.000706,-0.010434,0.001036,-0.001995,0.014008,0.013379,-0.005178,-0.001056,0.023820,-0.009765,-0.011110,0.006533,0.008507,0.008377,0.009765,0.002940,0.014587,-0.005887,0.016628,0.006199,0.003290,0.006825,-0.002866,0.019004,-0.019373,0.070650,0.010628,-0.004024,0.015908,-0.005744,0.027151,0.006490,0.008469,-0.003841,0.002400,...,0.005719,0.002859,0.015804,-0.008820,-0.015038,0.011919,0.021019,0.002284,-0.023381,0.027363,-0.005700,0.040488,0.006035,0.002035,0.042084,-0.016914,-0.020054,-0.000929,0.006236,0.011588,0.002281,-0.009697,0.000841,-0.002026,0.012584,0.009379,-0.022031,-0.026821,0.034061,-0.017483,0.039685,0.016050,0.006008,0.001627,0.013454,0.014296,-0.003207,0.014485,0.010015,0.013525
2020-01-03,-0.016056,-0.049502,0.000063,-0.009722,-0.009492,-0.012576,-0.011788,-0.012191,-0.001666,-0.007834,-0.017604,-0.001952,-0.002114,-0.015333,0.000394,-0.001070,-0.011483,-0.006935,-0.007728,0.004760,-0.002094,-0.004564,-0.014323,-0.011421,-0.018337,0.000089,-0.012361,-0.013260,-0.015916,-0.013170,-0.010183,0.001389,-0.006789,-0.012468,0.000481,-0.012139,-0.018904,-0.010093,-0.013261,-0.003927,...,-0.006286,-0.015206,-0.037948,-0.004344,0.007786,0.009728,0.021095,-0.006699,0.012059,-0.021792,-0.010647,-0.007659,-0.017230,0.000000,-0.015392,0.006287,0.017594,-0.006140,-0.017043,0.000245,0.009718,0.004257,-0.008828,0.001305,-0.017722,0.002636,-0.003436,0.001701,-0.014833,0.004809,-0.023020,-0.008039,-0.011066,-0.012998,0.005886,-0.003132,-0.002614,-0.011924,-0.014111,0.000149
2020-01-06,0.002956,-0.011935,-0.016434,0.007968,0.007892,0.014641,0.073253,0.005239,-0.006530,0.005726,-0.011749,-0.007823,0.001353,0.011733,0.000920,0.003320,0.011616,-0.002831,0.000779,0.000902,0.004617,0.003553,-0.001118,0.019398,-0.004782,0.002925,-0.001857,0.001598,-0.021565,-0.006673,-0.004321,0.001090,0.007674,0.003291,-0.000262,0.014886,0.010007,-0.009262,0.012025,0.002885,...,-0.003916,0.014717,0.000109,-0.001759,0.013028,0.002294,0.009259,0.027755,0.010338,0.033168,-0.002152,-0.004108,-0.009220,0.008632,-0.019192,0.000767,0.015206,-0.005990,0.009389,0.001224,0.005202,0.013141,-0.002036,-0.001448,-0.015230,0.000526,-0.009579,-0.002038,-0.001979,-0.001436,-0.020844,0.007678,0.005506,-0.015089,-0.006474,-0.000589,-0.005779,0.007655,-0.012186,-0.007677
2020-01-07,0.003065,-0.003660,-0.011862,-0.004703,-0.005705,-0.007156,0.007317,-0.005559,-0.021590,-0.000959,0.022751,-0.012046,-0.012104,0.002031,0.003677,0.000214,0.003495,-0.009464,-0.005642,-0.006761,-0.010757,0.030268,0.013149,-0.009864,-0.006306,-0.008572,-0.011973,0.002533,0.028886,-0.008637,-0.002893,0.005740,-0.009405,-0.012047,-0.021308,0.002092,0.009810,0.008170,-0.003029,-0.008486,...,-0.004939,0.001664,0.013110,-0.011280,-0.012113,0.008369,0.002318,-0.001071,-0.008325,0.014375,-0.011117,-0.000375,0.013544,-0.005034,0.067698,-0.003833,-0.006523,-0.008286,0.000339,-0.002053,0.000690,0.000000,-0.009265,-0.018274,-0.011182,-0.002562,0.004642,-0.006467,0.004745,-0.002077,0.022830,-0.008184,0.005299,0.001393,-0.003759,0.001768,-0.000879,-0.005969,-0.006070,0.003380
2020-01-08,0.009873,0.022777,-0.011488,0.016086,0.007087,0.009689,-0.009204,0.004077,0.001962,0.013438,0.009032,-0.011084,0.009397,0.013067,-0.001701,-0.002989,-0.001990,0.003057,0.011739,0.003782,0.001372,0.013304,-0.017258,0.010386,0.004835,0.002763,0.004258,0.016191,-0.000649,0.003872,-0.008705,0.002263,0.000756,0.008995,0.008673,-0.007809,0.010398,0.004130,0.026507,0.004110,...,-0.017018,-0.006171,0.003882,0.007986,-0.002119,0.009208,0.004626,0.032620,

In [7]:
seq_length = 20
dataX = []
dataY = []

turns = len(ret)-seq_length
for i in range(0,turns):
    start = i
    dataX.append(ret[start:start+seq_length])
    dataY.append(ret.iloc[start+seq_length])

In [8]:
train_size = int(len(dataX) * 0.75)
X_train, X_test = dataX[0:train_size], dataX[train_size:]
y_train, y_test = dataY[0:train_size], dataY[train_size:]

In [9]:
X_train_new = np.array(X_train).reshape(174,20,501)
y_train_new = np.array(y_train).reshape(174,1,501)
X_test_new = np.array(X_test).reshape(58,20,501)
y_test_new = np.array(y_test).reshape(58,1,501)

In [10]:
model = Sequential()
model.add(LSTM(units = 512, return_sequences = True, input_shape = (X_train_new.shape[1],501)))
model.add(Dropout(0.01))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 512, return_sequences = True))
model.add(Dropout(0.01))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 512, return_sequences = True))
model.add(Dropout(0.01))
# Adding a fourth LSTM layer 
model.add(LSTM(units = 512))
model.add(Dropout(0.01))
# Adding the output layer
model.add(Dense(units = 1))

opt = keras.optimizers.Adam(learning_rate=0.001)
# Compiling the RNN
model.compile(optimizer = opt, loss = 'mean_squared_error')

In [16]:
predictions_df = pd.DataFrame(columns=ret.columns)
for item in enumerate(ret.columns):
  y_test_new2 = y_test_new[:,:,item[0]].reshape(58,1,1)
  y_train_new2 = y_train_new[:,:,item[0]].reshape(174,1,1)
  model.fit(X_train_new, y_train_new2, epochs = 50, batch_size = 4,validation_data=(X_test_new,y_test_new2), verbose=0)
  predictions=model.predict(X_test_new)
  predictions_df[item[1]]=predictions[:,0]

In [17]:
from google.colab import files
predictions_df.to_csv('lstm_pred.csv') 
files.download('lstm_pred.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>